In [3]:
import pandas as pd

In [7]:
df=pd.read_pickle('industry_labed.pickle')

In [8]:
df

,날짜,제목,본문,링크,산업
26,2025-12-01 04:00:00,"이어지는 건설업 규제·처벌 강화…""사후 규제영향평가 도입해야""",최근 국회에서 규제와 처벌 강화를 골자로 하는 건설산업 관련 법안을 대거 입법하는 ...,https://n.news.naver.com/mnews/article/008/000...,건설
33,2025-12-01 04:06:00,유럽 흐르는 제네시스 마그마… 고성능 브랜드로 뜨겁게 경쟁,출범 10년을 맞은 제네시스가 '마그마'를 앞세워 유럽시장 공략에 나선다. 고성능 ...,https://n.news.naver.com/mnews/article/008/000...,자동차
35,2025-12-01 04:09:00,"'닥터나우 방지법' 처리 임박… 벤처업계 ""혁신 족쇄"" 호소",일명 '닥터나우 방지법'(약사법 개정안)이 12월2일 국회 본회의에서 처리될 전망이...,https://n.news.naver.com/mnews/article/008/000...,헬스케어
64,2025-12-01 05:01:00,"[백상논단] 생명공학기술, 철저한 검증과 관리 필요하다",[서울경제] 자기 자손은 유전적으로 완벽해 건강하게 장수하기를 바라는 마음은 모든...,https://n.news.naver.com/mnews/article/011/000...,헬스케어
85,2025-12-01 05:29:00,5년 만에 10배 ‘폭풍성장’ 테슬라...“수입차 점유율 사상 첫 20% 돌파 전망”,[파이낸셜뉴스] 테슬라의 누적 판매 대수가 5년 만에 10배 가까이 늘어나는 등 국...,https://n.news.naver.com/mnews/article/014/000...,자동차
...,...,...,...,...,...
81317,2025-12-21 18:29:00,압구정4구역 재건축 분양가 '국평 35억8천만원'... 290㎡ 펜트하우스는 211억,서울 핵심 재건축 단지인 '압구정4구역'의 펜트하우스(전용 290㎡) 조합원 분담금...,https://n.news.naver.com/mnews/article/014/000...,건설
81432,2025-12-21 19:08:00,[AI헬스케어] 정부 주도 AI의료 기술 개발...닥터앤서3.0 참여 기업은,[이데일리 김승권 기자] 정부 주도 의료 인공지능(AI) 사업 ‘닥터앤서’가 3.0...,https://n.news.naver.com/mnews/article/018/000...,헬스케어
81495,2025-12-21 20:52:00,살 뺄 때만 먹는 줄 알았는데…알고보니 '완전식품'인 고구마의 놀라운 효능 [건강UP],[서울경제] 몸·마음·생활… 무분별한 정보는 많고 건강해야 할 곳도 많습니다. ...,https://n.news.naver.com/mnews/article/011/000...,헬스케어
81512,2025-12-21 21:24:00,[미리보는 이데일리 신문]차세대 비만약 핵심기술 K바이오가 장악,[이데일리 박지애 기자] 다음은 22일자 이데일리 신문 주요 기사다. △1면 -...,https://n.news.naver.com/mnews/article/018/000...,헬스케어


In [9]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# 1. 실제 존재하는 한국어 감성 분석 모델로 교체
# jaehyeong 모델
model_name = "jaehyeong/koelectra-base-v3-generalized-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to("cuda")

def analyze_sentiment_batch(texts, batch_size=64):
    model.eval()
    all_scores = []
    
    # tqdm으로 진행률 표시
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size].astype(str).tolist() # 문자열 변환
        
        # 토큰화 및 GPU 전송
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to("cuda")
        
        with torch.no_grad():
            outputs = model(**inputs)
            # 확률값 계산 (Softmax)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            
            # 모델 출력 구조: [0: 부정 확률, 1: 긍정 확률]
            # 예: 긍정 0.9, 부정 0.1 이면 점수는 0.8
            batch_scores = (probs[:, 1] - probs[:, 0]).cpu().numpy()
            all_scores.extend(batch_scores)
            
    return all_scores

# 2. 데이터프레임 적용
# '본문' 컬럼 사용
df['sentiment_score'] = analyze_sentiment_batch(df['본문'])

# 결과 확인
print(df[['제목', 'sentiment_score']].head())

100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [02:17<00:00,  1.09it/s]

                                                 제목  sentiment_score
26               이어지는 건설업 규제·처벌 강화…"사후 규제영향평가 도입해야"         0.390355
33                 유럽 흐르는 제네시스 마그마… 고성능 브랜드로 뜨겁게 경쟁         0.980447
35                '닥터나우 방지법' 처리 임박… 벤처업계 "혁신 족쇄" 호소         0.869161
64                   [백상논단] 생명공학기술, 철저한 검증과 관리 필요하다         0.848842
85  5년 만에 10배 ‘폭풍성장’ 테슬라...“수입차 점유율 사상 첫 20% 돌파 전망”         0.967868


In [10]:
df.to_pickle("sentiment.pickle")

In [11]:
df

,날짜,제목,본문,링크,산업,sentiment_score
26,2025-12-01 04:00:00,"이어지는 건설업 규제·처벌 강화…""사후 규제영향평가 도입해야""",최근 국회에서 규제와 처벌 강화를 골자로 하는 건설산업 관련 법안을 대거 입법하는 ...,https://n.news.naver.com/mnews/article/008/000...,건설,0.390355
33,2025-12-01 04:06:00,유럽 흐르는 제네시스 마그마… 고성능 브랜드로 뜨겁게 경쟁,출범 10년을 맞은 제네시스가 '마그마'를 앞세워 유럽시장 공략에 나선다. 고성능 ...,https://n.news.naver.com/mnews/article/008/000...,자동차,0.980447
35,2025-12-01 04:09:00,"'닥터나우 방지법' 처리 임박… 벤처업계 ""혁신 족쇄"" 호소",일명 '닥터나우 방지법'(약사법 개정안)이 12월2일 국회 본회의에서 처리될 전망이...,https://n.news.naver.com/mnews/article/008/000...,헬스케어,0.869161
64,2025-12-01 05:01:00,"[백상논단] 생명공학기술, 철저한 검증과 관리 필요하다",[서울경제] 자기 자손은 유전적으로 완벽해 건강하게 장수하기를 바라는 마음은 모든...,https://n.news.naver.com/mnews/article/011/000...,헬스케어,0.848842
85,2025-12-01 05:29:00,5년 만에 10배 ‘폭풍성장’ 테슬라...“수입차 점유율 사상 첫 20% 돌파 전망”,[파이낸셜뉴스] 테슬라의 누적 판매 대수가 5년 만에 10배 가까이 늘어나는 등 국...,https://n.news.naver.com/mnews/article/014/000...,자동차,0.967868
...,...,...,...,...,...,...
81317,2025-12-21 18:29:00,압구정4구역 재건축 분양가 '국평 35억8천만원'... 290㎡ 펜트하우스는 211억,서울 핵심 재건축 단지인 '압구정4구역'의 펜트하우스(전용 290㎡) 조합원 분담금...,https://n.news.naver.com/mnews/article/014/000...,건설,0.601551
81432,2025-12-21 19:08:00,[AI헬스케어] 정부 주도 AI의료 기술 개발...닥터앤서3.0 참여 기업은,[이데일리 김승권 기자] 정부 주도 의료 인공지능(AI) 사업 ‘닥터앤서’가 3.0...,https://n.news.naver.com/mnews/article/018/000...,헬스케어,0.882646
81495,2025-12-21 20:52:00,살 뺄 때만 먹는 줄 알았는데…알고보니 '완전식품'인 고구마의 놀라운 효능 [건강UP],[서울경제] 몸·마음·생활… 무분별한 정보는 많고 건강해야 할 곳도 많습니다. ...,https://n.news.naver.com/mnews/article/011/000...,헬스케어,0.980757
81512,2025-12-21 21:24:00,[미리보는 이데일리 신문]차세대 비만약 핵심기술 K바이오가 장악,[이데일리 박지애 기자] 다음은 22일자 이데일리 신문 주요 기사다. △1면 -...,https://n.news.naver.com/mnews/article/018/000...,헬스케어,0.364674


In [16]:
df['날짜'] = pd.to_datetime(df['날짜'])

In [18]:
df['날짜'] = df['날짜'].dt.strftime('%Y-%m-%d')

In [13]:
df.to_pickle('sentiment.pickle')

,날짜,제목,본문,산업,sentiment_score
26,2025-12-01,"이어지는 건설업 규제·처벌 강화…""사후 규제영향평가 도입해야""",최근 국회에서 규제와 처벌 강화를 골자로 하는 건설산업 관련 법안을 대거 입법하는 ...,건설,0.390355
33,2025-12-01,유럽 흐르는 제네시스 마그마… 고성능 브랜드로 뜨겁게 경쟁,출범 10년을 맞은 제네시스가 '마그마'를 앞세워 유럽시장 공략에 나선다. 고성능 ...,자동차,0.980447
35,2025-12-01,"'닥터나우 방지법' 처리 임박… 벤처업계 ""혁신 족쇄"" 호소",일명 '닥터나우 방지법'(약사법 개정안)이 12월2일 국회 본회의에서 처리될 전망이...,헬스케어,0.869161
64,2025-12-01,"[백상논단] 생명공학기술, 철저한 검증과 관리 필요하다",[서울경제] 자기 자손은 유전적으로 완벽해 건강하게 장수하기를 바라는 마음은 모든...,헬스케어,0.848842
85,2025-12-01,5년 만에 10배 ‘폭풍성장’ 테슬라...“수입차 점유율 사상 첫 20% 돌파 전망”,[파이낸셜뉴스] 테슬라의 누적 판매 대수가 5년 만에 10배 가까이 늘어나는 등 국...,자동차,0.967868
...,...,...,...,...,...
81317,2025-12-21,압구정4구역 재건축 분양가 '국평 35억8천만원'... 290㎡ 펜트하우스는 211억,서울 핵심 재건축 단지인 '압구정4구역'의 펜트하우스(전용 290㎡) 조합원 분담금...,건설,0.601551
81432,2025-12-21,[AI헬스케어] 정부 주도 AI의료 기술 개발...닥터앤서3.0 참여 기업은,[이데일리 김승권 기자] 정부 주도 의료 인공지능(AI) 사업 ‘닥터앤서’가 3.0...,헬스케어,0.882646
81495,2025-12-21,살 뺄 때만 먹는 줄 알았는데…알고보니 '완전식품'인 고구마의 놀라운 효능 [건강UP],[서울경제] 몸·마음·생활… 무분별한 정보는 많고 건강해야 할 곳도 많습니다. ...,헬스케어,0.980757
81512,2025-12-21,[미리보는 이데일리 신문]차세대 비만약 핵심기술 K바이오가 장악,[이데일리 박지애 기자] 다음은 22일자 이데일리 신문 주요 기사다. △1면 -...,헬스케어,0.364674


In [18]:
df

,날짜,제목,본문,산업,sentiment_score
26,2025-12-01,"이어지는 건설업 규제·처벌 강화…""사후 규제영향평가 도입해야""",최근 국회에서 규제와 처벌 강화를 골자로 하는 건설산업 관련 법안을 대거 입법하는 ...,건설,0.390355
33,2025-12-01,유럽 흐르는 제네시스 마그마… 고성능 브랜드로 뜨겁게 경쟁,출범 10년을 맞은 제네시스가 '마그마'를 앞세워 유럽시장 공략에 나선다. 고성능 ...,자동차,0.980447
35,2025-12-01,"'닥터나우 방지법' 처리 임박… 벤처업계 ""혁신 족쇄"" 호소",일명 '닥터나우 방지법'(약사법 개정안)이 12월2일 국회 본회의에서 처리될 전망이...,헬스케어,0.869161
64,2025-12-01,"[백상논단] 생명공학기술, 철저한 검증과 관리 필요하다",[서울경제] 자기 자손은 유전적으로 완벽해 건강하게 장수하기를 바라는 마음은 모든...,헬스케어,0.848842
85,2025-12-01,5년 만에 10배 ‘폭풍성장’ 테슬라...“수입차 점유율 사상 첫 20% 돌파 전망”,[파이낸셜뉴스] 테슬라의 누적 판매 대수가 5년 만에 10배 가까이 늘어나는 등 국...,자동차,0.967868
...,...,...,...,...,...
81317,2025-12-21,압구정4구역 재건축 분양가 '국평 35억8천만원'... 290㎡ 펜트하우스는 211억,서울 핵심 재건축 단지인 '압구정4구역'의 펜트하우스(전용 290㎡) 조합원 분담금...,건설,0.601551
81432,2025-12-21,[AI헬스케어] 정부 주도 AI의료 기술 개발...닥터앤서3.0 참여 기업은,[이데일리 김승권 기자] 정부 주도 의료 인공지능(AI) 사업 ‘닥터앤서’가 3.0...,헬스케어,0.882646
81495,2025-12-21,살 뺄 때만 먹는 줄 알았는데…알고보니 '완전식품'인 고구마의 놀라운 효능 [건강UP],[서울경제] 몸·마음·생활… 무분별한 정보는 많고 건강해야 할 곳도 많습니다. ...,헬스케어,0.980757
81512,2025-12-21,[미리보는 이데일리 신문]차세대 비만약 핵심기술 K바이오가 장악,[이데일리 박지애 기자] 다음은 22일자 이데일리 신문 주요 기사다. △1면 -...,헬스케어,0.364674


In [19]:
df.to_pickle('sentiment.pickle')

,날짜,제목,본문,산업,sentiment_score
26,2025-12-01,"이어지는 건설업 규제·처벌 강화…""사후 규제영향평가 도입해야""",최근 국회에서 규제와 처벌 강화를 골자로 하는 건설산업 관련 법안을 대거 입법하는 ...,건설,0.390355
33,2025-12-01,유럽 흐르는 제네시스 마그마… 고성능 브랜드로 뜨겁게 경쟁,출범 10년을 맞은 제네시스가 '마그마'를 앞세워 유럽시장 공략에 나선다. 고성능 ...,자동차,0.980447
35,2025-12-01,"'닥터나우 방지법' 처리 임박… 벤처업계 ""혁신 족쇄"" 호소",일명 '닥터나우 방지법'(약사법 개정안)이 12월2일 국회 본회의에서 처리될 전망이...,헬스케어,0.869161
64,2025-12-01,"[백상논단] 생명공학기술, 철저한 검증과 관리 필요하다",[서울경제] 자기 자손은 유전적으로 완벽해 건강하게 장수하기를 바라는 마음은 모든...,헬스케어,0.848842
85,2025-12-01,5년 만에 10배 ‘폭풍성장’ 테슬라...“수입차 점유율 사상 첫 20% 돌파 전망”,[파이낸셜뉴스] 테슬라의 누적 판매 대수가 5년 만에 10배 가까이 늘어나는 등 국...,자동차,0.967868
...,...,...,...,...,...
81317,2025-12-21,압구정4구역 재건축 분양가 '국평 35억8천만원'... 290㎡ 펜트하우스는 211억,서울 핵심 재건축 단지인 '압구정4구역'의 펜트하우스(전용 290㎡) 조합원 분담금...,건설,0.601551
81432,2025-12-21,[AI헬스케어] 정부 주도 AI의료 기술 개발...닥터앤서3.0 참여 기업은,[이데일리 김승권 기자] 정부 주도 의료 인공지능(AI) 사업 ‘닥터앤서’가 3.0...,헬스케어,0.882646
81495,2025-12-21,살 뺄 때만 먹는 줄 알았는데…알고보니 '완전식품'인 고구마의 놀라운 효능 [건강UP],[서울경제] 몸·마음·생활… 무분별한 정보는 많고 건강해야 할 곳도 많습니다. ...,헬스케어,0.980757
81512,2025-12-21,[미리보는 이데일리 신문]차세대 비만약 핵심기술 K바이오가 장악,[이데일리 박지애 기자] 다음은 22일자 이데일리 신문 주요 기사다. △1면 -...,헬스케어,0.364674
